# Parameter definition
Choose all the settings for the database creation.

In [1]:
#TODO: Change absolute to relative paths for input and output dir.
import os

home = os.path.expanduser("~")

db_name= 'test_video_dataset' #'colab_main_dataset'
# input_dir = os.path.join(home,'image_datasets')
# output_dir = os.path.join(home,'sql_database')
input_dir = os.path.join(home,'Desktop', 'image_datasets')
output_dir = os.path.join(home,'Desktop','sql_database')
database_names=['lfw'] # , 'scface', 'forenface', 'enfsi', 'enfsi2015'
detector_names=['mtcnn', 'mtcnn_serfiq'] # 'dlib', 'mtcnn', 'mtcnn_serfiq'
embedding_model_names=["ArcFace"] # , "Dlib", "ArcFace",
attributes_to_update=['gender']  # 'gender', 'age', 'emotion', 'race'
quality_model_names = ['tface'] # 'confusion_score', 'ser_fiq',
save_in_drive = False # To work in google colab



## SERFIQ model

Create SERFIQ model.

In [2]:
from sql_face.serfiq import get_serfiq_model
serfiq = get_serfiq_model()

/home/andrea/anaconda3/envs/sql-face/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[18:19:02] ../src/nnvm/legacy_json_util.cc:208: Loading symbol saved by previous version v1.8.0. Attempting to upgrade...
[18:19:02] ../src/nnvm/legacy_json_util.cc:216: Symbol successfully upgraded!
/home/andrea/anaconda3/envs/sql-face/lib/python3.7/site-packages/mxnet/gluon/block.py:1784: UserWarning: Cannot decide type for the following arguments. Consider providing them as input:
	data: None
  input_sym_arg_type = in_param.infer_type()[0]
[18:19:03] ../src/nnvm/legacy_json_util.cc:208: Loading symbol saved by previous version v0.8.0. Attempting to upgrade...
[18:19:03] ../src/nnvm/legacy_json_util.cc:216: Symbol successfully upgraded!
[18:19:03] ../src/nnvm/legacy_json_util.cc:208: Loading symbol saved by previous ve

# Database creation

database = SQLDataBase(db_name=db_name,
                        database_names=database_names, # , 'scface', 'forenface', 'enfsi', 'enfsi2015'
                        detector_names=detector_names,
                        embedding_model_names=embedding_model_names,
                        quality_model_names=quality_model_names,
                        save_in_drive=save_in_drive                        
                        )

database.fill_db()
database.update_db(attributes_to_update=attributes_to_update, force_update=False)

In [3]:
from sql_face.sqldb import SQLDataBase

2022-11-16 18:19:03.941500: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-16 18:19:04.374723: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-16 18:19:05.426663: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/andrea/anaconda3/envs/sql-face/lib/python3.7/site-packages/cv2/../../lib64:/home/andrea/Documents/cuda_software/cuda/lib64:/usr/local/cuda-11.2/lib64
2022-11-16 18:19:05.426820: W tensorflow/stream_executor

In [4]:
database = SQLDataBase(db_name=db_name,
                        input_dir = input_dir,
                        output_dir_name = output_dir,
                        database_names = database_names,
                        detector_names = detector_names,
                        embedding_model_names = embedding_model_names,
                        quality_model_names = quality_model_names)

In [5]:
database.__dict__

{'db_name': 'test_video_dataset',
 'input_dir': '/home/andrea/Desktop/image_datasets',
 'save_in_drive': False,
 'output_dir': '/home/andrea/Desktop/sql_database',
 'session': <sqlalchemy.orm.session.Session at 0x7f6f112dd210>,
 'databases': [<sql_face.databases.LFW at 0x7f6f1126bfd0>],
 'detector_names': ['mtcnn', 'mtcnn_serfiq'],
 'embedding_model_names': ['ArcFace'],
 'quality_model_names': ['tface']}

In [6]:
database.create_tables(serfiq=serfiq)

Creating image record from LFW: 0it [00:00, ?it/s]
TRIM Creating Cropped Images for detector mtcnn:   0%|          | 0/5 [00:00<?, ?it/s]2022-11-16 18:19:08.741550: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-16 18:19:08.742436: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-16 18:19:08.742820: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-16 18:19:08.743625: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions

1/1 [==============================] - 0s 208ms/step


TRIM Creating Cropped Images for detector mtcnn:  20%|██        | 1/5 [00:11<00:44, 11.11s/it]

1/1 [==============================] - 0s 101ms/step


TRIM Creating Cropped Images for detector mtcnn:  40%|████      | 2/5 [00:12<00:15,  5.13s/it]

...

TRIM Creating Cropped Images for detector mtcnn:  60%|██████    | 3/5 [00:12<00:06,  3.16s/it]

1/1 [==============================] - 0s 108ms/step


TRIM Creating Cropped Images for detector mtcnn:  80%|████████  | 4/5 [00:13<00:02,  2.26s/it]

1/1 [==============================] - 0s 23ms/step


TRIM Creating Cropped Images for detector mtcnn: 100%|██████████| 5/5 [00:14<00:00,  2.89s/it]
TRIM Creating Cropped Images for detector mtcnn_serfiq:   0%|          | 0/5 [00:00<?, ?it/s][18:19:23] ../src/executor/graph_executor.cc:1991: Subgraph backend MKLDNN is activated.
[18:19:23] ../src/executor/graph_executor.cc:1991: Subgraph backend MKLDNN is activated.
[18:19:23] ../src/executor/graph_executor.cc:1991: Subgraph backend MKLDNN is activated.
[18:19:23] ../src/executor/graph_executor.cc:1991: Subgraph backend MKLDNN is activated.
[18:19:23] ../src/executor/graph_executor.cc:1991: Subgraph backend MKLDNN is activated.
[18:19:23] ../src/executor/graph_executor.cc:1991: Subgraph backend MKLDNN is activated.
[18:19:23] ../src/executor/graph_executor.cc:1991: Subgraph backend MKLDNN is activated.
[18:19:23] ../src/executor/graph_executor.cc:1991: Subgraph backend MKLDNN is activated.
[18:19:23] ../src/executor/graph_executor.cc:1991: Subgraph backend MKLDNN is activated.
[18:19:23] 

In [7]:
database.update_tables(attributes_to_update, serfiq = serfiq)

Update cropped images: 0it [00:00, ?it/s]
Computing embeddings:   0%|          | 0/15 [00:00<?, ?it/s]

1/1 [==============================] - 2s 2s/step


Computing embeddings:   7%|▋         | 1/15 [00:04<00:57,  4.13s/it]

1/1 [==============================] - 0s 30ms/step


Computing embeddings:  13%|█▎        | 2/15 [00:04<00:28,  2.18s/it]

...

TRIM Computing SER-FIQ quality:  60%|██████    | 3/5 [00:57<00:38, 19.10s/it]

1/1 [==============================] - 0s 23ms/step


TRIM Computing SER-FIQ quality:  80%|████████  | 4/5 [01:16<00:19, 19.19s/it]

1/1 [==============================] - 0s 21ms/step


TRIM Computing SER-FIQ quality: 100%|██████████| 5/5 [01:35<00:00, 19.17s/it]
